In [ ]:
import pandas as pd
import numpy as np
import re
import collections
import warnings
warnings.filterwarnings('ignore')
import nltk
nltk.download('punkt')
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize 
from nltk.util import ngrams
russian_stopwords = stopwords.words("russian")
mystem = Mystem() 
remove = ['да','не','нет','хорошо']
for w in remove:
    russian_stopwords.remove(w)
russian_stopwords.append('который')
import pymorphy2

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [5]:
def col_to_text(column):
    text = ''
    for answer in column:
        if answer != '':
            answer = re.sub(r'[0-9]+|\W+', ' ', str(answer))
            text += (answer+' ')
    return text

def preprocess_text(text):
    text = re.sub(r'[0-9]+|\W+', ' ', str(text))
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    #text = " ".join(tokens)
    
    return tokens
        
def syn_parse(text):
    text = re.sub(r'подымать|повышать|поднять|поднимать',r'увеличивать',text)
    text = re.sub(r'оклад|оплата|(заработная плата)|(з п)|(з пл)|(заработный плата)',r'зарплата',text)
    text = re.sub(r'снижать|сокращать',r'уменьшать',text)
    text = re.sub(r'маленький',r'низкий',text)
    text = re.sub(r'повышение',r'увеличение',text)
    text = re.sub(r'руководитель|директор|начальник|начальство',r'руководство',text)
    text = re.sub(r'организация',r'компания',text)
    text = re.sub(r'своевременный',r'стабильный',text)
    return text

def extract_most_common_words(text, n):    
    wordcount = {}
    for word in text.split():
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1    
    word_counter = collections.Counter(wordcount)
    for word, count in word_counter.most_common(n):
        print(word, ": ", count)
        
def extract_most_common_ngrams(ngr, text, n):        
    token = word_tokenize(text)
    n_grams = list(ngrams(token, ngr))
    for i in n_grams:
        if i[0] == i[1]:
            n_grams.remove(i)  
    n_gram_count = {}
    for n_gram in n_grams:
        if n_gram not in n_gram_count:
            n_gram_count[n_gram] = 1
        else:
            n_gram_count[n_gram] += 1
    n_gram_counter = collections.Counter(n_gram_count)
    for i in n_gram_counter.most_common(n):
        print(i)

def most_common_POS(text,pos,n):
    morph = pymorphy2.MorphAnalyzer()
    pos_list = []
    words = word_tokenize(text)
    for word in words:
        if  morph.parse(word)[0].tag.POS == pos:
            pos_list.append(word)
    pos_count = {}
    for p in pos_list:
        if p not in pos_count:
            pos_count[p] = 1
        else:
            pos_count[p] += 1
    pos_counter = collections.Counter(pos_count)
    for i in pos_counter.most_common(n):
        print(i)
        
def clean_text(text):
    text = re.sub(r'[0-9]+|\W+', ' ', str(text))
    return text

def extract_ngrams_in_range(ngr, text, left, right):        
    token = word_tokenize(text)
    n_grams = list(ngrams(token, ngr))
    for i in n_grams:
        if i[0] == i[1]:
            n_grams.remove(i)  
    n_gram_count = {}
    for n_gram in n_grams:
        if n_gram not in n_gram_count:
            n_gram_count[n_gram] = 1
        else:
            n_gram_count[n_gram] += 1
    n_gram_counter = collections.Counter(n_gram_count)
    for i in n_gram_counter.most_common():
        if left <= i[1] <= right:
            print(i)

### Что Вам нравится больше всего в компании?

In [3]:
file1 = pd.read_excel('Открыте вопросы Вовлеченность 2018.xlsx', sheet_name=0)
text1 = col_to_text(file1['Что Вам нравится больше всего в компании?'])
text1_prep = preprocess_text(text1)
text1_prep = syn_parse(text1_prep)

In [9]:
extract_most_common_ngrams(2,text1_prep,200)

(('условие', 'труд'), 596)
(('выплата', 'зарплата'), 384)
(('стабильный', 'зарплата'), 354)
(('коллектив', 'стабильность'), 313)
(('дружный', 'коллектив'), 302)
(('рабочий', 'место'), 281)
(('работа', 'коллектив'), 259)
(('стабильный', 'выплата'), 256)
(('зарплата', 'труд'), 252)
(('стабильность', 'коллектив'), 241)
(('график', 'работа'), 238)
(('новый', 'технология'), 214)
(('соц', 'пакет'), 206)
(('хороший', 'коллектив'), 192)
(('интересный', 'работа'), 186)
(('коллектив', 'работать'), 170)
(('поставлять', 'задача'), 158)
(('непосредственный', 'руководство'), 157)
(('компания', 'работа'), 156)
(('зарплата', 'коллектив'), 146)
(('продукт', 'услуга'), 134)
(('не', 'нравиться'), 132)
(('коллектив', 'руководство'), 130)
(('белый', 'зарплата'), 128)
(('услуга', 'компания'), 125)
(('наш', 'компания'), 121)
(('нравиться', 'коллектив'), 118)
(('работа', 'стабильность'), 118)
(('социальный', 'гарантия'), 117)
(('стабильность', 'зарплата'), 117)
(('коллектив', 'нравиться'), 116)
(('новый', 'пр

In [12]:
most_common_POS(text1_prep, 'NOUN', 200)

('коллектив', 4443)
('компания', 2901)
('работа', 2753)
('стабильность', 2587)
('зарплата', 2345)
('руководство', 1550)
('услуга', 1431)
('возможность', 1162)
('труд', 1069)
('развитие', 921)
('человек', 879)
('сотрудник', 842)
('условие', 833)
('задача', 779)
('коллега', 640)
('рабочий', 583)
('продукт', 569)
('выплата', 525)
('место', 520)
('процесс', 503)
('технология', 473)
('цель', 437)
('клиент', 419)
('отношение', 391)
('уровень', 372)
('ростелеком', 363)
('проект', 361)
('качество', 357)
('решение', 344)
('связь', 342)
('график', 339)
('команда', 314)
('время', 305)
('пакет', 299)
('год', 298)
('работник', 297)
('рынок', 278)
('увеличение', 254)
('интернет', 238)
('подразделение', 232)
('результат', 230)
('день', 228)
('выполнение', 220)
('рост', 217)
('обучение', 207)
('обеспечение', 203)
('вопрос', 198)
('мочь', 198)
('достижение', 192)
('отдел', 183)
('дом', 179)
('сервис', 176)
('поддержка', 176)
('т', 176)
('гарантия', 174)
('абонент', 174)
('реализация', 168)
('общение', 

### Какие препятствия Вы видите для достижения целей и задач компании?

In [6]:
file2 = pd.read_excel('Открыте вопросы Вовлеченность 2018.xlsx', sheet_name=1)
text2 = col_to_text(file2['Какие препятствия Вы видите для достижения целей и задач компании?'])
text2_prep = preprocess_text(text2)
text2_prep = syn_parse(text2_prep)

In [11]:
extract_most_common_ngrams(2,text2_prep,200)

(('не', 'видеть'), 2002)
(('низкий', 'зарплата'), 1219)
(('нет', 'препятствие'), 745)
(('препятствие', 'нет'), 640)
(('видеть', 'препятствие'), 627)
(('препятствие', 'не'), 589)
(('зарплата', 'труд'), 566)
(('уровень', 'зарплата'), 518)
(('низкий', 'уровень'), 476)
(('достижение', 'цель'), 379)
(('бизнес', 'процесс'), 371)
(('не', 'знать'), 342)
(('цель', 'задача'), 334)
(('компания', 'не'), 300)
(('сокращение', 'штат'), 249)
(('зарплата', 'не'), 244)
(('мотивация', 'сотрудник'), 210)
(('принятие', 'решение'), 208)
(('рабочий', 'место'), 202)
(('большой', 'количество'), 201)
(('зарплата', 'низкий'), 201)
(('никакой', 'препятствие'), 197)
(('сотрудник', 'не'), 196)
(('взаимодействие', 'подразделение'), 191)
(('видеть', 'не'), 188)
(('не', 'хватать'), 183)
(('задача', 'компания'), 175)
(('не', 'мочь'), 172)
(('поставлять', 'задача'), 172)
(('технический', 'блок'), 170)
(('т', 'д'), 169)
(('руководство', 'не'), 166)
(('затрудняться', 'отвечать'), 164)
(('качество', 'услуга'), 163)
(('ника

### Напишите то, что Вам кажется преимуществом компании как работодателя?

In [13]:
file3 = pd.read_excel('Выгрузка доопрос Вовлеченность 2018.xlsx', sheet_name=0)
text3 = col_to_text(file3['Напишите то, что Вам кажется преимуществом компании как работодателя?'])
text3_prep = preprocess_text(text3)
text3_prep = syn_parse(text3_prep)

In [16]:
extract_most_common_ngrams(2,text3_prep,200)

(('белый', 'зарплата'), 544)
(('соц', 'пакет'), 482)
(('условие', 'труд'), 431)
(('стабильный', 'зарплата'), 420)
(('выплата', 'зарплата'), 306)
(('зарплата', 'труд'), 247)
(('карьерный', 'рост'), 245)
(('стабильный', 'выплата'), 237)
(('крупный', 'компания'), 232)
(('официальный', 'трудоустройство'), 206)
(('социальный', 'гарантия'), 199)
(('социальный', 'пакет'), 191)
(('рабочий', 'место'), 189)
(('возможность', 'карьерный'), 142)
(('преимущество', 'компания'), 112)
(('зарплата', 'соц'), 109)
(('график', 'работа'), 106)
(('зарплата', 'стабильность'), 105)
(('стабильный', 'компания'), 98)
(('компания', 'работа'), 94)
(('зарплата', 'стабильный'), 90)
(('хороший', 'условие'), 90)
(('зарплата', 'социальный'), 88)
(('белый', 'зп'), 81)
(('компания', 'стабильность'), 78)
(('компания', 'стабильный'), 77)
(('тк', 'рф'), 76)
(('трудовой', 'законодательство'), 74)
(('увеличивать', 'зарплата'), 73)
(('стабильность', 'компания'), 73)
(('зарплата', 'возможность'), 71)
(('достойный', 'зарплата'), 

In [17]:
most_common_POS(text3_prep, 'NOUN', 200)

('зарплата', 2381)
('компания', 2140)
('стабильность', 1379)
('работа', 995)
('труд', 850)
('возможность', 813)
('услуга', 744)
('пакет', 706)
('условие', 582)
('сотрудник', 513)
('выплата', 496)
('преимущество', 454)
('рост', 381)
('развитие', 368)
('рабочий', 323)
('рынок', 322)
('гарантия', 312)
('руководство', 309)
('место', 294)
('соцпакет', 251)
('обучение', 248)
('коллектив', 247)
('трудоустройство', 239)
('работник', 234)
('соблюдение', 231)
('уровень', 210)
('продукт', 210)
('ростелеком', 208)
('льгота', 198)
('связь', 191)
('технология', 185)
('процесс', 181)
('график', 176)
('проект', 171)
('наличие', 171)
('день', 155)
('отпуск', 143)
('качество', 137)
('работодатель', 133)
('клиент', 131)
('т', 131)
('задержка', 131)
('увеличение', 130)
('время', 129)
('законодательство', 124)
('рф', 123)
('предоставление', 117)
('задача', 108)
('опыт', 107)
('офис', 105)
('программа', 102)
('человек', 102)
('год', 99)
('спектр', 98)
('надежность', 95)
('обеспечение', 95)
('специалист', 93

### Были ли в последние 6 месяцев события, которые повлияли на Ваше отношение к компании? Приведите примеры таких событий?

In [18]:
file4 = pd.read_excel('Выгрузка доопрос Вовлеченность 2018.xlsx', sheet_name=1)
text4 = col_to_text(file4['Были ли в последние 6 месяцев события, которые повлияли на Ваше отношение к компании? Приведите примеры таких событий?'])
text4_prep = preprocess_text(text4)
text4_prep = syn_parse(text4_prep)

In [19]:
extract_most_common_ngrams(2,text4_prep,200)

(('кафетерий', 'льгота'), 429)
(('событие', 'не'), 367)
(('нет', 'не'), 287)
(('отношение', 'компания'), 286)
(('увеличение', 'зарплата'), 253)
(('коллективный', 'договор'), 215)
(('компания', 'не'), 174)
(('сокращение', 'штат'), 174)
(('не', 'нет'), 164)
(('не', 'событие'), 160)
(('зарплата', 'не'), 133)
(('зарплата', 'труд'), 131)
(('индексация', 'зарплата'), 118)
(('объем', 'работа'), 116)
(('последний', 'месяц'), 113)
(('введение', 'кафетерий'), 108)
(('нет', 'событие'), 104)
(('смена', 'руководство'), 98)
(('рабочий', 'место'), 98)
(('новый', 'коллективный'), 97)
(('сокращение', 'персонал'), 86)
(('сокращение', 'сотрудник'), 79)
(('новый', 'год'), 77)
(('не', 'мочь'), 77)
(('это', 'не'), 75)
(('низкий', 'зарплата'), 74)
(('не', 'происходить'), 71)
(('сотрудник', 'не'), 70)
(('повлиять', 'отношение'), 70)
(('постоянный', 'сокращение'), 63)
(('не', 'сокращение'), 61)
(('не', 'да'), 60)
(('ничто', 'не'), 60)
(('не', 'увеличиваться'), 59)
(('руководство', 'не'), 59)
(('работа', 'не'),

###  Что важно улучшить в Вашем подразделении?

In [21]:
file5 = pd.read_excel('Выгрузка доопрос Вовлеченность 2018.xlsx', sheet_name=2)
text5 = col_to_text(file5['Что важно улучшить в Вашем подразделении?'])
text5_prep = preprocess_text(text5)
text5_prep = syn_parse(text5_prep)

In [22]:
extract_most_common_ngrams(2,text5_prep,200)

(('увеличивать', 'зарплата'), 1062)
(('зарплата', 'труд'), 851)
(('наш', 'подразделение'), 346)
(('условие', 'труд'), 286)
(('увеличение', 'зарплата'), 274)
(('рабочий', 'место'), 239)
(('зарплата', 'увеличивать'), 225)
(('уровень', 'зарплата'), 213)
(('увеличивать', 'штат'), 173)
(('труд', 'увеличивать'), 140)
(('подразделение', 'хорошо'), 126)
(('бизнес', 'процесс'), 124)
(('увеличивать', 'уровень'), 120)
(('объем', 'работа'), 113)
(('взаимодействие', 'подразделение'), 104)
(('зарплата', 'сотрудник'), 102)
(('не', 'хватать'), 92)
(('труд', 'зарплата'), 90)
(('зарплата', 'не'), 87)
(('сотрудник', 'увеличивать'), 82)
(('не', 'знать'), 82)
(('карьерный', 'рост'), 77)
(('количество', 'сотрудник'), 75)
(('увеличивать', 'количество'), 74)
(('рабочий', 'процесс'), 74)
(('размер', 'зарплата'), 70)
(('зарплата', 'работник'), 66)
(('штат', 'сотрудник'), 65)
(('мотивация', 'сотрудник'), 64)
(('затрудняться', 'отвечать'), 64)
(('поставлять', 'задача'), 64)
(('увеличивать', 'зп'), 63)
(('ничто', 

In [23]:
most_common_POS(text5_prep, 'NOUN', 200)

('зарплата', 3304)
('сотрудник', 1430)
('подразделение', 1400)
('труд', 1279)
('работа', 1276)
('руководство', 622)
('работник', 591)
('компания', 570)
('процесс', 569)
('увеличение', 556)
('рабочий', 500)
('уровень', 488)
('условие', 436)
('штат', 404)
('взаимодействие', 387)
('место', 367)
('задача', 362)
('отношение', 343)
('человек', 317)
('мотивация', 287)
('специалист', 277)
('возможность', 258)
('количество', 251)
('персонал', 247)
('обеспечение', 240)
('отдел', 240)
('обучение', 228)
('время', 220)
('т', 217)
('ремонт', 210)
('материал', 209)
('связь', 208)
('обязанность', 207)
('оборудование', 206)
('год', 200)
('система', 196)
('вопрос', 190)
('помещение', 178)
('выполнение', 176)
('качество', 172)
('услуга', 165)
('функционал', 159)
('нагрузка', 158)
('улучшение', 156)
('коллектив', 150)
('клиент', 146)
('рост', 146)
('бизнес', 145)
('блок', 138)
('ответственность', 137)
('инструмент', 137)
('распределение', 134)
('решение', 128)
('филиал', 125)
('изменение', 125)
('сокращен

### После 1 месяца

In [38]:
file6 = pd.read_excel('Опрос сотрудников 1 месяц работы в компании.xlsx')
file6 = file6.fillna('')
text6 = col_to_text(file6['Unnamed: 14'][3:])
text6_prep = preprocess_text(text6)
text6_prep = syn_parse(text6_prep)

In [56]:
extract_most_common_ngrams(2,text6_prep,200)

(('новый', 'сотрудник'), 41)
(('затрудняться', 'отвечать'), 40)
(('рабочий', 'место'), 32)
(('данный', 'момент'), 31)
(('пока', 'не'), 29)
(('не', 'знать'), 22)
(('не', 'мочь'), 22)
(('бизнес', 'процесс'), 19)
(('увеличивать', 'зарплата'), 18)
(('пока', 'устраивать'), 12)
(('работа', 'программа'), 10)
(('устраивать', 'не'), 10)
(('не', 'готовый'), 10)
(('момент', 'устраивать'), 10)
(('отвечать', 'вопрос'), 9)
(('нет', 'предложение'), 9)
(('т', 'д'), 9)
(('обучение', 'новый'), 8)
(('не', 'хватать'), 8)
(('пока', 'затрудняться'), 8)
(('увеличивать', 'уровень'), 8)
(('весь', 'отлично'), 8)
(('работа', 'комфортный'), 7)
(('данный', 'этап'), 7)
(('момент', 'не'), 7)
(('комфортный', 'работа'), 7)
(('готовый', 'отвечать'), 7)
(('пока', 'нет'), 7)
(('мочь', 'сказать'), 7)
(('весь', 'устраивать'), 6)
(('рабочий', 'день'), 6)
(('хорошо', 'пока'), 6)
(('план', 'адаптация'), 6)
(('пока', 'сложно'), 5)
(('сложно', 'сказать'), 5)
(('предложение', 'нет'), 5)
(('зарплата', 'труд'), 5)
(('сотрудник', '

In [84]:
extract_ngrams_in_range(2,text6_prep,1,2)

(('провожать', 'обучение'), 2)
(('упрощать', 'работа'), 2)
(('нужно', 'открывать'), 2)
(('очень', 'программа'), 2)
(('программа', 'сделать'), 2)
(('затрудняться', 'ответ'), 2)
(('бесплатный', 'кофе'), 2)
(('пока', 'незнать'), 2)
(('не', 'уровень'), 2)
(('стаж', 'работа'), 2)
(('сотрудник', 'офис'), 2)
(('питьевой', 'вода'), 2)
(('бесплатный', 'питание'), 2)
(('постановка', 'задача'), 2)
(('задача', 'хорошо'), 2)
(('ремонт', 'помещение'), 2)
(('денежный', 'довольствие'), 2)
(('пока', 'хорошо'), 2)
(('это', 'сделать'), 2)
(('пока', 'отвечать'), 2)
(('устраивать', 'данный'), 2)
(('знать', 'вводить'), 2)
(('вводить', 'crm'), 2)
(('crm', 'система'), 2)
(('отвечать', 'пока'), 2)
(('работник', 'компания'), 2)
(('компания', 'хотя'), 2)
(('работа', 'сделать'), 2)
(('работа', 'пока'), 2)
(('отвечать', 'затрудняться'), 2)
(('каждый', 'сотрудник'), 2)
(('начало', 'рабочий'), 2)
(('горячий', 'вода'), 2)
(('организовывать', 'столовая'), 2)
(('работа', 'направление'), 2)
(('проводить', 'ремонт'), 2)


(('система', 'например'), 1)
(('например', 'r'), 1)
(('r', 'понятный'), 1)
(('понятный', 'написать'), 1)
(('написать', 'человеческий'), 1)
(('человеческий', 'язык'), 1)
(('язык', 'указываться'), 1)
(('указываться', 'конкретный'), 1)
(('конкретный', 'человек'), 1)
(('человек', 'обращаться'), 1)
(('обращаться', 'разъяснение'), 1)
(('разъяснение', 'почему'), 1)
(('почему', 'система'), 1)
(('система', 'заливать'), 1)
(('заливать', 'иной'), 1)
(('иной', 'данные'), 1)
(('данные', 'просто'), 1)
(('просто', 'предлагать'), 1)
(('предлагать', 'свой'), 1)
(('свой', 'помощь'), 1)
(('помощь', 'дело'), 1)
(('дело', 'приходить'), 1)
(('приходить', 'опыт'), 1)
(('опыт', 'работа'), 1)
(('не', 'каждый'), 1)
(('каждый', 'человек'), 1)
(('человек', 'особенно'), 1)
(('особенно', 'новенький'), 1)
(('новенький', 'мочь'), 1)
(('мочь', 'открыто'), 1)
(('открыто', 'написать'), 1)
(('написать', 'портал'), 1)
(('портал', 'идея'), 1)
(('идея', 'хорошо'), 1)
(('хорошо', 'прорабатывать'), 1)
(('прорабатывать', 'сист

(('место', 'общий'), 1)
(('пользование', 'бесплатный'), 1)
(('бесплатный', 'парковка'), 1)
(('парковка', 'авто'), 1)
(('авто', 'сотрудник'), 1)
(('сотрудник', 'также'), 1)
(('также', 'преимущество'), 1)
(('преимущество', 'повторяться'), 1)
(('повторяться', 'подарок'), 1)
(('сотрудник', 'нга'), 1)
(('нга', 'привязка'), 1)
(('привязка', 'выслуга'), 1)
(('выслуга', 'год'), 1)
(('год', 'дмс'), 1)
(('дмс', 'прохождение'), 1)
(('прохождение', 'испытательный'), 1)
(('срок', 'это'), 1)
(('это', 'базовый'), 1)
(('базовый', 'вещь'), 1)
(('вещь', 'должный'), 1)
(('должный', 'крупный'), 1)
(('компания', 'экономить'), 1)
(('экономить', 'не'), 1)
(('не', 'стоить'), 1)
(('стоить', 'ничто'), 1)
(('ничто', 'введение'), 1)
(('введение', 'расширенный'), 1)
(('расширенный', 'дмс'), 1)
(('дмс', 'нужно'), 1)
(('нужно', 'время'), 1)
(('время', 'проводить'), 1)
(('проводить', 'анализарплатаока'), 1)
(('анализарплатаока', 'очень'), 1)
(('хорошо', 'понятный'), 1)
(('понятный', 'система'), 1)
(('система', 'мотив

(('отвечать', 'следующий'), 1)
(('следующий', 'устраивать'), 1)
(('хватать', 'чистота'), 1)
(('чистота', 'офис'), 1)
(('офис', 'уборная'), 1)
(('уборная', 'сотрудник'), 1)
(('сотрудник', 'обращаться'), 1)
(('обращаться', 'корректный'), 1)
(('корректный', 'общаться'), 1)
(('общаться', 'взаимодействие'), 1)
(('отдел', 'условие'), 1)
(('условие', 'отдых'), 1)
(('отдых', 'график'), 1)
(('график', 'перерыв'), 1)
(('перерыв', 'разрабатывать'), 1)
(('разрабатывать', 'прозрачный'), 1)
(('прозрачный', 'матрица'), 1)
(('матрица', 'взаимодействие'), 1)
(('взаимодействие', 'нужно'), 1)
(('нужно', 'открытый'), 1)
(('открытый', 'очень'), 1)
(('приходиться', 'делать'), 1)
(('делать', 'заявка'), 1)
(('заявка', 'техподдержка'), 1)
(('техподдержка', 'поставлять'), 1)
(('поставлять', 'кулер'), 1)
(('кулер', 'вода'), 1)
(('вода', 'кухня'), 1)
(('кухня', 'слух'), 1)
(('слух', 'фильтр'), 1)
(('фильтр', 'стоять'), 1)
(('стоять', 'очень'), 1)
(('очень', 'давно'), 1)
(('давно', 'не'), 1)
(('не', 'меняться'), 1

(('биометрия', 'замечание'), 1)
(('нет', 'оперативный'), 1)
(('решение', 'поставлять'), 1)
(('задача', 'весь'), 1)
(('программа', 'сводить'), 1)
(('сводить', 'ничто'), 1)
(('ничто', 'увеличивать'), 1)
(('зп', 'улучшать'), 1)
(('улучшать', 'увеличивать'), 1)
(('увеличивать', 'поставка'), 1)
(('поставка', 'расходный'), 1)
(('материал', 'интсталлятор'), 1)
(('интсталлятор', 'хорошо'), 1)
(('хорошо', 'затрудняться'), 1)
(('количество', 'система'), 1)
(('система', 'пока'), 1)
(('вопрос', 'вникать'), 1)
(('вникать', 'продукт'), 1)
(('продукт', 'действовать'), 1)
(('действовать', 'программа'), 1)
(('программа', 'очень'), 1)
(('сложный', 'процесс'), 1)
(('процесс', 'доступ'), 1)
(('система', 'необходимо'), 1)
(('необходимо', 'отрабатывать'), 1)
(('отрабатывать', 'легкий'), 1)
(('легкий', 'процедура'), 1)
(('получение', 'логинов'), 1)
(('логинов', 'пароль'), 1)
(('пароль', 'долгий'), 1)
(('долгий', 'работа'), 1)
(('работа', 'поддержка'), 1)
(('поддержка', 'починять'), 1)
(('починять', 'связь'),

(('несколько', 'главный'), 1)
(('главный', 'задача'), 1)
(('задача', 'закреплять'), 1)
(('закреплять', 'наставник'), 1)
(('наставник', 'четкий'), 1)
(('четкий', 'план'), 1)
(('план', 'развитие'), 1)
(('развитие', 'новый'), 1)
(('сотрудник', 'предоставление'), 1)
(('предоставление', 'промежуточный'), 1)
(('промежуточный', 'итоговый'), 1)
(('итоговый', 'обратный'), 1)
(('связь', 'максимально'), 1)
(('максимально', 'обеспечивать'), 1)
(('обеспечивать', 'новичок'), 1)
(('новичок', 'актуальный'), 1)
(('актуальный', 'обучать'), 1)
(('материал', 'продукт'), 1)
(('продукт', 'процесс'), 1)
(('процесс', 'тариф'), 1)
(('тариф', 'пр'), 1)
(('пр', 'научать'), 1)
(('научать', 'пользоваться'), 1)
(('пользоваться', 'упрощать'), 1)
(('упрощать', 'система'), 1)
(('система', 'crm'), 1)
(('crm', 'ускорять'), 1)
(('ускорять', 'процесс'), 1)
(('процесс', 'прохождение'), 1)
(('прохождение', 'этап'), 1)
(('этап', 'arm'), 1)
(('arm', 'слишком'), 1)
(('слишком', 'долгий'), 1)
(('долгий', 'время'), 1)
(('время',

### После 2 месяцев

In [86]:
file7 = pd.read_excel('Опрос сотрудников 2 месяца работы в компании.xlsx')
file7 = file7.fillna('')
text7 = col_to_text(file7['Unnamed: 14'][3:])
text7_prep = preprocess_text(text7)
text7_prep = syn_parse(text7_prep)

In [88]:
extract_most_common_ngrams(2,text7_prep,200)

(('увеличивать', 'зарплата'), 36)
(('затрудняться', 'отвечать'), 34)
(('новый', 'сотрудник'), 29)
(('данный', 'момент'), 23)
(('рабочий', 'место'), 22)
(('пока', 'не'), 21)
(('не', 'знать'), 20)
(('весь', 'устраивать'), 14)
(('бизнес', 'процесс'), 10)
(('не', 'хватать'), 10)
(('работа', 'программа'), 10)
(('увеличение', 'зарплата'), 8)
(('данный', 'этап'), 8)
(('взаимодействие', 'подразделение'), 8)
(('зарплата', 'увеличивать'), 8)
(('сложно', 'сказать'), 7)
(('не', 'мочь'), 7)
(('предложение', 'нет'), 7)
(('прием', 'пища'), 7)
(('весь', 'отлично'), 7)
(('увеличивать', 'зп'), 7)
(('график', 'работа'), 6)
(('отлично', 'увеличивать'), 6)
(('условие', 'труд'), 6)
(('количество', 'программа'), 6)
(('пока', 'устраивать'), 6)
(('зарплата', 'труд'), 6)
(('удаленный', 'работа'), 5)
(('устраивать', 'пока'), 5)
(('сотрудник', 'хорошо'), 5)
(('гибкий', 'график'), 5)
(('момент', 'устраивать'), 5)
(('не', 'готовый'), 5)
(('должностной', 'обязанность'), 5)
(('уменьшать', 'количество'), 5)
(('это', '

In [89]:
extract_ngrams_in_range(2,text7_prep,1,2)

(('работа', 'сотрудник'), 2)
(('улучшение', 'не'), 2)
(('не', 'требоваться'), 2)
(('нет', 'устраивать'), 2)
(('оптимизировать', 'работа'), 2)
(('момент', 'достаточно'), 2)
(('достаточно', 'комфортно'), 2)
(('нет', 'ответ'), 2)
(('процесс', 'построить'), 2)
(('программа', 'работать'), 2)
(('свой', 'обязанность'), 2)
(('офис', 'гончарный'), 2)
(('работа', 'комфортно'), 2)
(('работа', 'уровень'), 2)
(('уровень', 'зп'), 2)
(('работа', 'ничто'), 2)
(('ничто', 'устраивать'), 2)
(('зарплата', 'данный'), 2)
(('устраивать', 'ничто'), 2)
(('устраивать', 'комфортно'), 2)
(('возможность', 'удаленный'), 2)
(('программа', 'r'), 2)
(('каждый', 'этаж'), 2)
(('очень', 'комфортный'), 2)
(('труд', 'устраивать'), 2)
(('срок', 'работа'), 2)
(('обеденный', 'зона'), 2)
(('вполне', 'комфортный'), 2)
(('разный', 'подразделение'), 2)
(('компания', 'данный'), 2)
(('не', 'соответствовать'), 2)
(('ремонт', 'мебель'), 2)
(('сотрудник', 'приводить'), 2)
(('рекомендация', 'нет'), 2)
(('нет', 'данный'), 2)
(('ход', 'р

(('хорошо', 'это'), 1)
(('это', 'отражать'), 1)
(('отражать', 'понятно'), 1)
(('понятно', 'портал'), 1)
(('портал', 'ускорять'), 1)
(('ускорять', 'выдача'), 1)
(('выдача', 'телефон'), 1)
(('телефон', 'стационарный'), 1)
(('стационарный', 'рабочий'), 1)
(('рабочий', 'хорошо'), 1)
(('хорошо', 'помыть'), 1)
(('помыть', 'этаж'), 1)
(('этаж', 'включая'), 1)
(('включая', 'туалет'), 1)
(('туалет', 'очень'), 1)
(('очень', 'грязно'), 1)
(('грязно', 'выполнять'), 1)
(('обещание', 'данные'), 1)
(('данные', 'трудоустройство'), 1)
(('трудоустройство', 'ранее'), 1)
(('ранее', 'предлагать'), 1)
(('предлагать', 'комната'), 1)
(('отдых', 'устанавливать'), 1)
(('устанавливать', 'турник'), 1)
(('турник', 'ввиду'), 1)
(('ввиду', 'характер'), 1)
(('работа', 'именно'), 1)
(('именно', 'час'), 1)
(('час', 'сидеть'), 1)
(('сидеть', 'компьютер'), 1)
(('компьютер', 'отличный'), 1)
(('отличный', 'разнообразие'), 1)
(('разнообразие', 'вариант'), 1)
(('вариант', 'растяжка'), 1)
(('растяжка', 'спина'), 1)
(('спина',

(('комфортно', 'сделать'), 1)
(('сделать', 'офис'), 1)
(('офис', 'современный'), 1)
(('современный', 'затрудняться'), 1)
(('отвечать', 'гибкий'), 1)
(('график', 'предпраздничный'), 1)
(('предпраздничный', 'день'), 1)
(('день', 'сделать'), 1)
(('кабинет', 'пока'), 1)
(('отвечать', 'итак'), 1)
(('итак', 'отлично'), 1)
(('зарплата', 'сегодняшний'), 1)
(('сегодняшний', 'день'), 1)
(('день', 'устраивать'), 1)
(('обновлять', 'офисный'), 1)
(('офисный', 'мебель'), 1)
(('мебель', 'упрощенный'), 1)
(('упрощенный', 'поиск'), 1)
(('поиск', 'r'), 1)
(('r', 'работа'), 1)
(('работа', 'есэд'), 1)
(('есэд', 'очень'), 1)
(('очень', 'медленный'), 1)
(('медленный', 'атмосфера'), 1)
(('атмосфера', 'коллектив'), 1)
(('коллектив', 'налаживать'), 1)
(('подразделение', 'обеспечивать'), 1)
(('обеспечивать', 'подразделение'), 1)
(('подразделение', 'сканер'), 1)
(('сканер', 'оказывать'), 1)
(('оказывать', 'информационный'), 1)
(('информационный', 'поддержка'), 1)
(('поддержка', 'вовлеченность'), 1)
(('вовлеченно

(('облегчать', 'увеличивать'), 1)
(('зарплата', 'супер'), 1)
(('супер', 'организовывать'), 1)
(('внутренний', 'обучение'), 1)
(('адаптация', 'уравнивать'), 1)
(('уравнивать', 'kpi'), 1)
(('kpi', 'разный'), 1)
(('подразделение', 'не'), 1)
(('знать', 'автоматизация'), 1)
(('процесс', 'рабочий'), 1)
(('сотрудник', 'затрудняться'), 1)
(('отвечать', 'скорость'), 1)
(('oracle', 'есэд'), 1)
(('есэд', 'устанавливать'), 1)
(('место', 'мощный'), 1)
(('мощный', 'компьютер'), 1)
(('компьютер', 'обучение'), 1)
(('обучение', 'время'), 1)
(('адаптация', 'прикреплять'), 1)
(('прикреплять', 'новичок'), 1)
(('новичок', 'куратор'), 1)
(('куратор', 'увеличение'), 1)
(('зарплата', 'достаточно'), 1)
(('достаточно', 'пока'), 1)
(('знать', 'не'), 1)
(('отлично', 'большой'), 1)
(('монитор', 'хотеться'), 1)
(('хотеться', 'комфортабельный'), 1)
(('комфортабельный', 'кресло'), 1)
(('не', 'комментарий'), 1)
(('комментарий', 'тренинг'), 1)
(('тренинг', 'карьерный'), 1)
(('карьерный', 'рост'), 1)
(('рост', 'парковка

(('сотрудник', 'хождение'), 1)
(('хождение', 'кабинет'), 1)
(('кабинет', 'собирание'), 1)
(('собирание', 'подпись'), 1)
(('подпись', 'увеличивать'), 1)
(('зарплата', 'парковка'), 1)
(('парковка', 'увеличение'), 1)
(('затрудняться', 'пока'), 1)
(('пока', 'отвечать'), 1)
(('отвечать', 'открыто'), 1)
(('открыто', 'обсуждать'), 1)
(('обсуждать', 'эскалировать'), 1)
(('эскалировать', 'проблема'), 1)
(('проблема', 'ничто'), 1)
(('ничто', 'обновлять'), 1)
(('обновлять', 'парк'), 1)
(('парк', 'персональный'), 1)
(('персональный', 'компьютер'), 1)
(('компьютер', 'новый'), 1)
(('сотрудник', 'условие'), 1)
(('условие', 'комфортный'), 1)
(('комфортный', 'хорошо'), 1)
(('хорошо', 'разрабатывать'), 1)
(('сотрудник', 'максимальный'), 1)
(('максимальный', 'погружение'), 1)
(('процесс', 'kpi'), 1)
(('kpi', 'гибкий'), 1)
(('работа', 'многий'), 1)
(('многий', 'сотрудник'), 1)
(('сотрудник', 'сталкиваться'), 1)
(('сталкиваться', 'проблема'), 1)
(('проблема', 'проезд'), 1)
(('проезд', 'работа'), 1)
(('рабо

(('часто', 'аргус'), 1)
(('аргус', 'бис'), 1)
(('бис', 'есуз'), 1)
(('есуз', 'например'), 1)
(('например', 'сипало'), 1)
(('сипало', 'смарттуб'), 1)
(('смарттуб', 'корректно'), 1)
(('корректно', 'работать'), 1)
(('работать', 'личный'), 1)
(('личный', 'впечатление'), 1)
(('впечатление', 'весь'), 1)
(('весь', 'смена'), 1)
(('смена', 'вводить'), 1)
(('вводить', 'технический'), 1)
(('технический', 'перерыв'), 1)
(('перерыв', 'плавать'), 1)
(('плавать', 'начало'), 1)
(('начало', 'рабочий'), 1)
(('не', 'фиксировать'), 1)
(('фиксировать', 'договоренность'), 1)
(('договоренность', 'именно'), 1)
(('именно', 'плавать'), 1)
(('плавать', 'сегодня'), 1)
(('сегодня', 'приходить'), 1)
(('приходить', 'завтра'), 1)
(('завтра', 'это'), 1)
(('не', 'требовать'), 1)
(('требовать', 'согласование'), 1)
(('согласование', 'главное'), 1)
(('главное', 'сотрудник'), 1)
(('сотрудник', 'контролировать'), 1)
(('контролировать', 'свой'), 1)
(('свой', 'участие'), 1)
(('участие', 'совещание'), 1)
(('совещание', 'учитыв

### После 3 месяцев

In [90]:
file8 = pd.read_excel('Опрос сотрудников 3 месяца работы в компании.xlsx')
file8 = file8.fillna('')
text8 = col_to_text(file8['Unnamed: 14'][3:])
text8_prep = preprocess_text(text8)
text8_prep = syn_parse(text8_prep)

In [92]:
extract_most_common_ngrams(2,text8_prep,200)

(('увеличивать', 'зарплата'), 40)
(('рабочий', 'место'), 30)
(('новый', 'сотрудник'), 29)
(('затрудняться', 'отвечать'), 28)
(('весь', 'устраивать'), 19)
(('не', 'мочь'), 15)
(('данный', 'момент'), 15)
(('пока', 'не'), 12)
(('не', 'хватать'), 12)
(('уровень', 'зарплата'), 11)
(('зарплата', 'труд'), 11)
(('не', 'знать'), 10)
(('рабочий', 'процесс'), 9)
(('увеличение', 'зарплата'), 8)
(('зарплата', 'увеличивать'), 8)
(('т', 'д'), 8)
(('пока', 'устраивать'), 8)
(('бизнес', 'процесс'), 8)
(('отвечать', 'вопрос'), 7)
(('прием', 'пища'), 7)
(('комфортный', 'работа'), 7)
(('отвечать', 'давать'), 6)
(('давать', 'вопрос'), 6)
(('нет', 'предложение'), 6)
(('работа', 'компания'), 6)
(('информационный', 'система'), 6)
(('адаптация', 'новый'), 6)
(('работа', 'программа'), 6)
(('работа', 'комфортный'), 6)
(('увеличивать', 'зп'), 6)
(('комната', 'отдых'), 6)
(('рабочий', 'время'), 5)
(('удаленный', 'доступ'), 5)
(('это', 'очень'), 5)
(('обучение', 'сотрудник'), 5)
(('мочь', 'отвечать'), 5)
(('уменьша

In [94]:
extract_ngrams_in_range(2,text8_prep,1,2)

(('пока', 'рано'), 2)
(('месяц', 'не'), 2)
(('обеденный', 'зона'), 2)
(('очень', 'здорово'), 2)
(('сотрудник', 'т'), 2)
(('ответственный', 'сотрудник'), 2)
(('хотеться', 'обучение'), 2)
(('очень', 'комфортный'), 2)
(('объем', 'работа'), 2)
(('полный', 'мера'), 2)
(('принимать', 'участие'), 2)
(('организовывать', 'система'), 2)
(('программа', 'работа'), 2)
(('нет', 'пока'), 2)
(('некоторый', 'процесс'), 2)
(('сложно', 'отвечать'), 2)
(('оборудование', 'рабочий'), 2)
(('адаптационный', 'мероприятие'), 2)
(('близко', 'познакомиться'), 2)
(('задавать', 'вопрос'), 2)
(('корпоративный', 'культура'), 2)
(('отношение', 'сотрудник'), 2)
(('процесс', 'адаптация'), 2)
(('никто', 'коллега'), 2)
(('уменьшать', 'бюрократический'), 2)
(('текучка', 'кадр'), 2)
(('работа', 'хотеться'), 2)
(('устраивать', 'офис'), 2)
(('офис', 'обслуживание'), 2)
(('систематизировать', 'рабочий'), 2)
(('процесс', 'хорошо'), 2)
(('работа', 'сделать'), 2)
(('сделать', 'ремонт'), 2)
(('ремонт', 'офис'), 2)
(('офис', 'доста

(('приходиться', 'часто'), 1)
(('часто', 'работать'), 1)
(('ничто', 'упрощать'), 1)
(('упрощать', 'документооборот'), 1)
(('документооборот', 'хорошо'), 1)
(('хорошо', 'зарплата'), 1)
(('комфортный', 'незнать'), 1)
(('незнать', 'попросить'), 1)
(('попросить', 'уборщица'), 1)
(('уборщица', 'вымывать'), 1)
(('вымывать', 'комната'), 1)
(('пища', 'доплачивать'), 1)
(('доплачивать', 'это'), 1)
(('мочь', 'подробно'), 1)
(('подробно', 'описывать'), 1)
(('описывать', 'момент'), 1)
(('момент', 'трудоустройство'), 1)
(('трудоустройство', 'допускать'), 1)
(('допускать', 'письмо'), 1)
(('письмо', 'почта'), 1)
(('почта', 'описывать'), 1)
(('описывать', 'пункт'), 1)
(('пункт', 'необходимо'), 1)
(('выполнять', 'течение'), 1)
(('течение', 'испытательный'), 1)
(('срок', 'первый'), 1)
(('например', 'курсы'), 1)
(('курсы', 'проходить'), 1)
(('проходить', 'документ'), 1)
(('документ', 'сдавать'), 1)
(('сдавать', 'подписывать'), 1)
(('подписывать', 'придавать'), 1)
(('придавать', 'уверенность'), 1)
(('увер

(('очень', 'обидно'), 1)
(('обидно', 'наш'), 1)
(('наш', 'малыш'), 1)
(('малыш', 'оставлять'), 1)
(('оставлять', 'новогодний'), 1)
(('подарок', 'самый'), 1)
(('самый', 'главное'), 1)
(('главное', 'это'), 1)
(('это', 'ребенок'), 1)
(('ребенок', 'весь'), 1)
(('весь', 'компания'), 1)
(('компания', 'давать'), 1)
(('давать', 'новогодний'), 1)
(('подарок', 'организовывать'), 1)
(('организовывать', 'елка'), 1)
(('елка', 'проводить'), 1)
(('проводить', 'тренинг'), 1)
(('тренинг', 'онлайн'), 1)
(('онлайн', 'обучение'), 1)
(('обучение', 'увеличивать'), 1)
(('не', 'соответствовать'), 1)
(('соответствовать', 'обязанность'), 1)
(('обязанность', 'проводить'), 1)
(('проводить', 'различный'), 1)
(('различный', 'тренинг'), 1)
(('тренинг', 'курсы'), 1)
(('курсы', 'увеличение'), 1)
(('квалификация', 'обсуждать'), 1)
(('обсуждать', 'руководство'), 1)
(('руководство', 'зарплата'), 1)
(('зарплата', 'обеспечение'), 1)
(('обеспечение', 'спецодежда'), 1)
(('спецодежда', 'технический'), 1)
(('технический', 'док

(('пк', 'p'), 1)
(('p', 's'), 1)
(('s', 'пк'), 1)
(('пк', 'рабочий'), 1)
(('место', 'достаточно'), 1)
(('достаточно', 'производительный'), 1)
(('производительный', 'поэтому'), 1)
(('работать', 'довольно'), 1)
(('комфортно', 'думать'), 1)
(('думать', 'организовывать'), 1)
(('организовывать', 'нормально'), 1)
(('нормально', 'просто'), 1)
(('просто', 'любой'), 1)
(('любой', 'специалист'), 1)
(('специалист', 'необходимо'), 1)
(('необходимо', 'время'), 1)
(('время', 'адаптация'), 1)
(('место', 'вводить'), 1)
(('вводить', 'некий'), 1)
(('некий', 'инструкция'), 1)
(('инструкция', 'давать'), 1)
(('давать', 'понимание'), 1)
(('понимание', 'функция'), 1)
(('функция', 'задача'), 1)
(('задача', 'соседний'), 1)
(('соседний', 'подразделение'), 1)
(('подразделение', 'увеличивать'), 1)
(('зарплата', 'специалист'), 1)
(('специалист', 'предоставлять'), 1)
(('отдельный', 'помещение'), 1)
(('помещение', 'прием'), 1)
(('пища', 'стабильный'), 1)
(('стабильный', 'размер'), 1)
(('зарплата', 'мало'), 1)
(('мал

(('транспорт', 'нет'), 1)
(('предложение', 'доработка'), 1)
(('доработка', 'организационно'), 1)
(('организационно', 'функциональный'), 1)
(('функциональный', 'работать'), 1)
(('весь', 'страна'), 1)
(('страна', 'увеличение'), 1)
(('сделать', 'жизнь'), 1)
(('жизнь', 'комфортный'), 1)
(('комфортный', 'хороший'), 1)
(('хороший', 'считать'), 1)
(('считать', 'хороший'), 1)
(('хороший', 'указывать'), 1)
(('указывать', 'опрос'), 1)
(('опрос', 'очень'), 1)
(('очень', 'напрягать'), 1)
(('напрягать', 'вылетание'), 1)
(('вылетание', 'рабочий'), 1)
(('программа', 'час'), 1)
(('час', 'работа'), 1)
(('работа', 'случайный'), 1)
(('случайный', 'интервал'), 1)
(('интервал', 'время'), 1)
(('время', 'стокнуться'), 1)
(('стокнуться', 'нереагирование'), 1)
(('нереагирование', 'рабочий'), 1)
(('место', 'загрузка'), 1)
(('загрузка', 'согласование'), 1)
(('согласование', 'старший'), 1)
(('старший', 'смена'), 1)
(('смена', 'перезагружать'), 1)
(('перезагружать', 'пк'), 1)
(('пк', 'нажатие'), 1)
(('нажатие', 'к

(('адекватный', 'обучение'), 1)
(('сотрудник', 'обновлять'), 1)
(('обновлять', 'оргтехника'), 1)
(('оргтехника', 'долго'), 1)
(('долго', 'печально'), 1)
(('печально', 'думать'), 1)
(('думать', 'производить'), 1)
(('производить', 'косметический'), 1)
(('место', 'кабинет'), 1)
(('кабинет', 'закрытый'), 1)
(('закрытый', 'кабинет'), 1)
(('кабинет', 'слишком'), 1)
(('слишком', 'шумно'), 1)
(('шумно', 'мешать'), 1)
(('мешать', 'сосредотачиваться'), 1)
(('сосредотачиваться', 'быстро'), 1)
(('быстро', 'согласовывать'), 1)
(('согласовывать', 'документ'), 1)
(('документ', 'недостаточно'), 1)
(('недостаточно', 'проходить'), 1)
(('проходить', 'время'), 1)
(('время', 'предложение'), 1)
(('предложение', 'сегодня'), 1)
(('сегодня', 'комфортный'), 1)
(('сделать', 'достаточно'), 1)
(('достаточно', 'пока'), 1)
(('пока', 'дополнительный'), 1)
(('дополнительный', 'предложение'), 1)
(('взгляд', 'работа'), 1)
(('работа', 'налаживать'), 1)
(('налаживать', 'достаточно'), 1)
(('комфортно', 'говорить'), 1)
(('г

### После 1 недели

In [101]:
file9 = pd.read_excel('Опрос новых сотрудников 1 неделя работы в компании.xlsx')

In [104]:
file9 = file9.fillna('')

In [108]:
q = [
       'Как прошла Ваша первая неделя в Ростелеком?',
       'Почему Вы пришли в Ростелеком?',
       'Что нам следует улучшить, для того чтобы работа у нас стала более комфортной?',
             ]
cols = file9.columns
corp = []
for col in cols:
    corp.append(preprocess_text(col_to_text(file9[col])))

d = dict(zip(q,corp))
key_list = list(d)

In [112]:
for key in d:
    print(key)
    extract_most_common_words(d[key],100)
    print('')

Как прошла Ваша первая неделя в Ростелеком?
хорошо :  233
отлично :  181
проходить :  129
неделя :  115
работа :  94
новый :  92
коллектив :  90
первый :  87
очень :  82
коллега :  74
продуктивно :  74
рабочий :  72
информация :  55
познакомиться :  52
знакомство :  50
не :  46
процесс :  45
задача :  39
интересный :  37
место :  34
компания :  34
весь :  34
получать :  32
обучение :  32
доступ :  31
плодотворно :  30
интересно :  30
изучать :  28
познавательный :  27
замечательно :  27
насыщенно :  26
узнавать :  25
свой :  23
программа :  22
помогать :  22
комфортно :  21
нормально :  21
быстро :  21
знакомиться :  19
вопрос :  19
день :  18
обязанность :  18
достаточно :  18
изучение :  17
руководитель :  17
необходимый :  17
инструктаж :  17
время :  17
сотрудник :  16
продукт :  16
начинать :  16
адаптация :  16
знание :  16
позитивно :  14
успешно :  13
ростелеком :  13
ознакомление :  13
принимать :  13
это :  13
большой :  12
погружение :  12
спокойно :  12
безопасность :  12
т

In [113]:
for key in d:
    print(key)
    extract_most_common_ngrams(2,d[key],50)
    print('')

Как прошла Ваша первая неделя в Ростелеком?
(('неделя', 'проходить'), 64)
(('первый', 'неделя'), 62)
(('хорошо', 'отлично'), 37)
(('рабочий', 'место'), 31)
(('новый', 'информация'), 27)
(('отлично', 'хорошо'), 24)
(('проходить', 'хорошо'), 20)
(('очень', 'хорошо'), 18)
(('узнавать', 'новый'), 17)
(('знакомство', 'коллега'), 17)
(('рабочий', 'процесс'), 16)
(('познакомиться', 'коллега'), 16)
(('знакомство', 'коллектив'), 16)
(('познакомиться', 'коллектив'), 16)
(('продуктивно', 'хорошо'), 11)
(('новый', 'интересный'), 11)
(('проходить', 'обучение'), 11)
(('охрана', 'труд'), 10)
(('отлично', 'очень'), 8)
(('отлично', 'первый'), 8)
(('отлично', 'продуктивно'), 8)
(('хорошо', 'продуктивно'), 8)
(('неделя', 'работа'), 8)
(('новый', 'знание'), 8)
(('большой', 'объем'), 7)
(('ростелеком', 'проходить'), 7)
(('первый', 'день'), 7)
(('проходить', 'отлично'), 7)
(('хорошо', 'первый'), 7)
(('хорошо', 'новый'), 7)
(('проходить', 'успешно'), 6)
(('проходить', 'интересный'), 6)
(('отлично', 'познаком

In [114]:
extract_ngrams_in_range(2,d[key],1,3)

(('ответ', 'давать'), 3)
(('не', 'сталкиваться'), 3)
(('слишком', 'маленький'), 3)
(('замечание', 'нет'), 3)
(('не', 'очень'), 3)
(('полностью', 'устраивать'), 3)
(('оптимизировать', 'работа'), 3)
(('целое', 'весь'), 3)
(('просто', 'замечательно'), 3)
(('высокий', 'уровень'), 3)
(('хорошо', 'ничто'), 3)
(('ничто', 'не'), 3)
(('не', 'улучшать'), 3)
(('пока', 'отвечать'), 3)
(('пока', 'мало'), 3)
(('сотрудник', 'работа'), 3)
(('качество', 'работа'), 3)
(('компания', 'ничто'), 3)
(('ничто', 'затрудняться'), 3)
(('весь', 'доступ'), 3)
(('момент', 'время'), 3)
(('момент', 'сложно'), 3)
(('этап', 'не'), 3)
(('приступать', 'работа'), 3)
(('пока', 'рано'), 3)
(('рано', 'говорить'), 3)
(('пока', 'ничто'), 3)
(('комфортно', 'пока'), 3)
(('сотрудник', 'пока'), 3)
(('работа', 'ресурс'), 3)
(('плата', 'пока'), 3)
(('организация', 'рабочий'), 3)
(('сей', 'пора'), 3)
(('не', 'иметь'), 3)
(('комната', 'отдых'), 3)
(('нет', 'замечание'), 3)
(('отвечать', 'хорошо'), 3)
(('чай', 'кофе'), 3)
(('сегодняшни

(('руководитель', 'вывести'), 1)
(('вывести', 'таракан'), 1)
(('таракан', 'пп'), 1)
(('пп', 'настоящий'), 1)
(('время', 'сложно'), 1)
(('отвечать', 'материально'), 1)
(('материально', 'денежный'), 1)
(('денежный', 'стимулирование'), 1)
(('стимулирование', 'пока'), 1)
(('улучшать', 'неделя'), 1)
(('неделя', 'работать'), 1)
(('работать', 'совместный'), 1)
(('совместный', 'групповой'), 1)
(('групповой', 'обучение'), 1)
(('обучение', 'продолжать'), 1)
(('продолжать', 'активно'), 1)
(('активно', 'трансформация'), 1)
(('трансформация', 'бизнес'), 1)
(('бизнес', 'корпоративный'), 1)
(('корпоративный', 'культура'), 1)
(('культура', 'улучшать'), 1)
(('улучшать', 'ситуация'), 1)
(('ситуация', 'парковка'), 1)
(('парковка', 'не'), 1)
(('вопрос', 'прекрасно'), 1)
(('прекрасно', 'отличный'), 1)
(('отличный', 'график'), 1)
(('график', 'прекрасный'), 1)
(('прекрасный', 'коллектив'), 1)
(('не', 'мало'), 1)
(('мало', 'важно'), 1)
(('важно', 'иметься'), 1)
(('иметься', 'комната'), 1)
(('комната', 'эмоцио

(('отдельный', 'единица'), 1)
(('единица', 'позиция'), 1)
(('позиция', 'наставник'), 1)
(('наставник', 'слишком'), 1)
(('компания', 'неделя'), 1)
(('неделя', 'предложение'), 1)
(('предложение', 'улудшение'), 1)
(('улудшение', 'пока'), 1)
(('хорошо', 'кресло'), 1)
(('кресло', 'комфортный'), 1)
(('комфортный', 'персонал'), 1)
(('персонал', 'закупать'), 1)
(('закупать', 'ничто'), 1)
(('ускорять', 'предоставление'), 1)
(('системный', 'приложение'), 1)
(('приложение', 'устраивать'), 1)
(('устраивать', 'сделать'), 1)
(('сделать', 'дресс'), 1)
(('код', 'свободный'), 1)
(('свободный', 'отличный'), 1)
(('отличный', 'рассматривать'), 1)
(('рассматривать', 'вариант'), 1)
(('вариант', 'адаптация'), 1)
(('адаптация', 'внутри'), 1)
(('внутри', 'департамент'), 1)
(('департамент', 'типо'), 1)
(('типо', 'велком'), 1)
(('велком', 'внутри'), 1)
(('внутри', 'подразделение'), 1)
(('подразделение', 'знакомить'), 1)
(('знакомить', 'коллега'), 1)
(('коллега', 'топ'), 1)
(('топ', 'горячий'), 1)
(('горячий', 'н

(('предоставлять', 'весь'), 1)
(('доступ', 'нужный'), 1)
(('программа', 'устройство'), 1)
(('устройство', 'единообразно'), 1)
(('единообразно', 'сложно'), 1)
(('этап', 'отлично'), 1)
(('весь', 'довольный'), 1)
(('идеально', 'выдача'), 1)
(('выдача', 'современный'), 1)
(('современный', 'оборудование'), 1)
(('оборудование', 'сотрудник'), 1)
(('сотрудник', 'автоматический'), 1)
(('автоматический', 'заведение'), 1)
(('заведение', 'необходимый'), 1)
(('хорошо', 'предложение'), 1)
(('нет', 'устанавливать'), 1)
(('устанавливать', 'корпоративный'), 1)
(('корпоративный', 'чат'), 1)
(('чат', 'быстрый'), 1)
(('быстрый', 'решение'), 1)
(('решение', 'задача'), 1)
(('задача', 'добавлять'), 1)
(('добавлять', 'справчоник'), 1)
(('справчоник', 'сотрудник'), 1)
(('сотрудник', 'вн'), 1)
(('вн', 'телефон'), 1)
(('телефон', 'весь'), 1)
(('сотрудник', 'некоторый'), 1)
(('некоторый', 'отсутствовать'), 1)
(('отсутствовать', 'организовывать'), 1)
(('организовывать', 'вай'), 1)
(('вай', 'фай'), 1)
(('фай', 'каб

(('оформление', 'налаживать'), 1)
(('налаживать', 'процесс'), 1)
(('процесс', 'план'), 1)
(('адаптация', 'не'), 1)
(('готовый', 'пока'), 1)
(('прорабатывать', 'количество'), 1)
(('количество', 'время'), 1)
(('ничто', 'подымать'), 1)
(('плата', 'проводить'), 1)
(('проводить', 'часто'), 1)
(('часто', 'тренинг'), 1)
(('тренинг', 'обучение'), 1)
(('обучение', 'устраивать'), 1)
(('развивать', 'престижность'), 1)
(('престижность', 'работа'), 1)
(('работа', 'ростелеком'), 1)
(('ростелеком', 'число'), 1)
(('число', 'увеличение'), 1)
(('увеличение', 'заработный'), 1)
(('плата', 'соблюдать'), 1)
(('соблюдать', 'договоренность'), 1)
(('договоренность', 'режим'), 1)
(('режим', 'работа'), 1)
(('работа', 'обговариваться'), 1)
(('обговариваться', 'этап'), 1)
(('собеседование', 'полностью'), 1)
(('полностью', 'проговаривать'), 1)
(('проговаривать', 'обращаться'), 1)
(('обращаться', 'административный'), 1)
(('административный', 'вопрос'), 1)
(('вопрос', 'случай'), 1)
(('случай', 'проходить'), 1)
(('про

### Классификация комментариев

In [63]:
data2 = pd.read_excel('binary.xlsx')

C этим файлом хотел модель попробовать но количество документов в корпусе 36000, что заставит стеммер столько же количество раз запускаться, что крайне долго. Поэтому взглянем просто на ответы

In [59]:
positive = data2[data2.label == 1]
pos_text = col_to_text(positive.text)
pos_text_prep = preprocess_text(pos_text)
pos_text_prep = syn_parse(pos_text_prep)
extract_most_common_ngrams(2,pos_text_prep,200)

(('стабильный', 'зарплата'), 1646)
(('интересный', 'работа'), 1462)
(('соц', 'пакет'), 1435)
(('карьерный', 'рост'), 1277)
(('стабильный', 'компания'), 813)
(('хороший', 'коллектив'), 806)
(('белый', 'зарплата'), 736)
(('дружный', 'коллектив'), 660)
(('развивающийся', 'компания'), 638)
(('возможность', 'карьерный'), 578)
(('крупный', 'компания'), 571)
(('компания', 'стабильный'), 499)
(('выплата', 'зарплата'), 465)
(('стабильный', 'выплата'), 440)
(('условие', 'труд'), 418)
(('компания', 'стабильность'), 402)
(('стабильный', 'заработок'), 393)
(('работа', 'стабильный'), 379)
(('зарплата', 'труд'), 365)
(('стабильность', 'стабильный'), 358)
(('социальный', 'пакет'), 340)
(('зарплата', 'стабильность'), 336)
(('возможность', 'развитие'), 334)
(('новый', 'технология'), 307)
(('социальный', 'гарантия'), 297)
(('коллектив', 'стабильный'), 295)
(('официальный', 'трудоустройство'), 293)
(('компания', 'возможность'), 289)
(('коллектив', 'стабильность'), 277)
(('работа', 'стабильность'), 276)
((

In [62]:
negative = data2[data2.label == 0]
neg_text = col_to_text(negative.text)
neg_text_prep = preprocess_text(neg_text)
neg_text_prep = syn_parse(neg_text_prep)
extract_most_common_ngrams(2,neg_text_prep,200)

(('низкий', 'зарплата'), 3332)
(('уровень', 'зарплата'), 1537)
(('зарплата', 'низкий'), 1378)
(('низкий', 'уровень'), 992)
(('зарплата', 'труд'), 928)
(('не', 'мешать'), 538)
(('ничто', 'не'), 493)
(('зарплата', 'не'), 489)
(('карьерный', 'рост'), 429)
(('объем', 'работа'), 321)
(('низкий', 'зп'), 251)
(('не', 'высокий'), 247)
(('зарплата', 'ничто'), 228)
(('постоянный', 'сокращение'), 220)
(('зарплата', 'отсутствие'), 220)
(('очень', 'низкий'), 217)
(('бизнес', 'процесс'), 213)
(('высокий', 'зарплата'), 205)
(('невысокий', 'зарплата'), 198)
(('не', 'соответствовать'), 197)
(('труд', 'низкий'), 192)
(('зарплата', 'нет'), 189)
(('уровень', 'зп'), 188)
(('рабочий', 'место'), 183)
(('ничто', 'низкий'), 179)
(('сокращение', 'штат'), 160)
(('большой', 'объем'), 158)
(('зарплата', 'уровень'), 151)
(('работа', 'не'), 149)
(('работа', 'низкий'), 140)
(('завтрашний', 'день'), 134)
(('зп', 'низкий'), 133)
(('зарплата', 'большой'), 131)
(('увеличение', 'зарплата'), 124)
(('компания', 'не'), 121)


Здесь относительно немного комментариев, поэтому удалось дождаться препроцессинга

In [3]:
from lightgbm import LGBMClassifier
from scipy.sparse import csr_matrix

In [10]:
data = pd.read_excel('classification.xlsx')

In [11]:
pd.set_option('display.max_colwidth', -1)

In [12]:
data.head()

,comment,description,label
0,До конца декабря не было информации о страховании родственников. Только личное обращение прояснило ситуацию. Личное обращение было отработано оперативно и с готовностью помочь.,Непрозрачность недостатки организации процессов,0
1,сложные и долгие подсчеты,Отсутствие ожидаемого результата,4
2,согласование изменений по сотрудникам занимает вечность,Долгая обработка запроса,2
3,очень непрозрачный процесс,Долгая обработка запроса,2
4,Требуется выравнивание социальных программ между Ростелекомом и ДЗО,Отсутствие ожидаемого результата,4


In [5]:
data.label.value_counts()

2    307
0    297
4    285
1    274
3    264
Name: label, dtype: int64

In [24]:
corpus1 = []
for line in data.comment:
    corpus1.append(preprocess_text(line))

In [31]:
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(data.comment).toarray()
y = data.label

In [9]:
def bow(classifier,X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))
    print(accuracy_score(y_test, y_pred))

In [33]:
bow(RandomForestClassifier(),X_vect,y)

[[54  0  0  0  0]
 [ 0 50  0  0  0]
 [ 0  1 68  0  4]
 [ 0  0  0 47  0]
 [ 0  1 10  0 51]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       0.96      1.00      0.98        50
           2       0.87      0.93      0.90        73
           3       1.00      1.00      1.00        47
           4       0.93      0.82      0.87        62

    accuracy                           0.94       286
   macro avg       0.95      0.95      0.95       286
weighted avg       0.94      0.94      0.94       286

0.9440559440559441


In [34]:
bow(GradientBoostingClassifier(),X_vect,y)

[[54  0  0  0  0]
 [ 0 50  0  0  0]
 [ 0  1 67  0  5]
 [ 0  0  0 47  0]
 [ 2  0  7  0 53]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        54
           1       0.98      1.00      0.99        50
           2       0.91      0.92      0.91        73
           3       1.00      1.00      1.00        47
           4       0.91      0.85      0.88        62

    accuracy                           0.95       286
   macro avg       0.95      0.95      0.95       286
weighted avg       0.95      0.95      0.95       286

0.9475524475524476


In [35]:
bow(LGBMClassifier(),X_vect,y)

[[54  0  0  0  0]
 [ 0 46  1  0  3]
 [ 0  2 63  0  8]
 [ 0  0  0 47  0]
 [ 5  1  5  3 48]]
              precision    recall  f1-score   support

           0       0.92      1.00      0.96        54
           1       0.94      0.92      0.93        50
           2       0.91      0.86      0.89        73
           3       0.94      1.00      0.97        47
           4       0.81      0.77      0.79        62

    accuracy                           0.90       286
   macro avg       0.90      0.91      0.91       286
weighted avg       0.90      0.90      0.90       286

0.9020979020979021


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer()
X_tfidf = tfidfconverter.fit_transform(data.comment).toarray()

In [26]:
bow(RandomForestClassifier(),X_tfidf,y)

[[54  0  0  0  0]
 [ 0 50  0  0  0]
 [ 0  1 68  0  4]
 [ 0  0  0 47  0]
 [ 0  2  3  0 57]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       0.94      1.00      0.97        50
           2       0.96      0.93      0.94        73
           3       1.00      1.00      1.00        47
           4       0.93      0.92      0.93        62

    accuracy                           0.97       286
   macro avg       0.97      0.97      0.97       286
weighted avg       0.97      0.97      0.96       286

0.965034965034965


In [28]:
bow(GradientBoostingClassifier(),X_tfidf,y)

[[54  0  0  0  0]
 [ 0 50  0  0  0]
 [ 2  1 66  2  2]
 [ 0  0  0 47  0]
 [ 1  1  7  2 51]]
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        54
           1       0.96      1.00      0.98        50
           2       0.90      0.90      0.90        73
           3       0.92      1.00      0.96        47
           4       0.96      0.82      0.89        62

    accuracy                           0.94       286
   macro avg       0.94      0.95      0.94       286
weighted avg       0.94      0.94      0.94       286

0.9370629370629371


In [27]:
bow(LGBMClassifier(),X_tfidf,y)

[[54  0  0  0  0]
 [ 0 47  0  0  3]
 [ 0  1 67  0  5]
 [ 0  0  0 47  0]
 [ 2  2 11  1 46]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        54
           1       0.94      0.94      0.94        50
           2       0.86      0.92      0.89        73
           3       0.98      1.00      0.99        47
           4       0.85      0.74      0.79        62

    accuracy                           0.91       286
   macro avg       0.92      0.92      0.92       286
weighted avg       0.91      0.91      0.91       286

0.9125874125874126


In [3]:
data2 = pd.read_excel('binary.xlsx')

In [4]:
data2.head()

,text,label
0,единственный крупный оператор связи в Чувашии.,1
1,Стабильная зарплата,1
2,"официальное трудоустройство, стабильность, гар...",1
3,"Удобно, понятно, доступно.",1
4,"Современная компания. Отличная практика, хорош...",1


In [5]:
data2.tail()

,text,label
36322,низкая зарплата,0
36323,низкая зарплата,0
36324,низкая зарплата,0
36325,Очень большая нагрузка,0
36326,Не мешает ни чего. Обязательно порекомендую.,0


In [5]:
vectorizer = CountVectorizer()
X_vect = vectorizer.fit_transform(data2.text).toarray()
y = data2.label

In [6]:
X_vect = csr_matrix(X_vect, dtype = 'float')

In [7]:
X_vect

<36327x22081 sparse matrix of type '<class 'numpy.float64'>'
	with 263541 stored elements in Compressed Sparse Row format>

In [67]:
bow(RandomForestClassifier(),X_vect,y)

[[2932  151]
 [ 217 3966]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3083
           1       0.96      0.95      0.96      4183

    accuracy                           0.95      7266
   macro avg       0.95      0.95      0.95      7266
weighted avg       0.95      0.95      0.95      7266

0.9493531516652904


In [68]:
bow(GradientBoostingClassifier(),X_vect,y)

[[2423  660]
 [ 178 4005]]
              precision    recall  f1-score   support

           0       0.93      0.79      0.85      3083
           1       0.86      0.96      0.91      4183

    accuracy                           0.88      7266
   macro avg       0.90      0.87      0.88      7266
weighted avg       0.89      0.88      0.88      7266

0.8846683181943298


In [10]:
bow(LGBMClassifier(),X_vect,y)

[[2970  113]
 [ 305 3878]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      3083
           1       0.97      0.93      0.95      4183

    accuracy                           0.94      7266
   macro avg       0.94      0.95      0.94      7266
weighted avg       0.94      0.94      0.94      7266

0.9424717864024222


In [15]:
tfidfconverter = TfidfVectorizer()
X_tfidf = tfidfconverter.fit_transform(data2.text).toarray()

In [16]:
X_tfidf = csr_matrix(X_tfidf, dtype = 'float')

In [17]:
bow(RandomForestClassifier(),X_tfidf,y)

[[2974  109]
 [ 222 3961]]
              precision    recall  f1-score   support

           0       0.93      0.96      0.95      3083
           1       0.97      0.95      0.96      4183

    accuracy                           0.95      7266
   macro avg       0.95      0.96      0.95      7266
weighted avg       0.96      0.95      0.95      7266

0.9544453619598128


In [18]:
bow(GradientBoostingClassifier(),X_tfidf,y)

[[2402  681]
 [ 149 4034]]
              precision    recall  f1-score   support

           0       0.94      0.78      0.85      3083
           1       0.86      0.96      0.91      4183

    accuracy                           0.89      7266
   macro avg       0.90      0.87      0.88      7266
weighted avg       0.89      0.89      0.88      7266

0.8857693366363887


In [19]:
bow(LGBMClassifier(),X_tfidf,y)

[[2972  111]
 [ 285 3898]]
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      3083
           1       0.97      0.93      0.95      4183

    accuracy                           0.95      7266
   macro avg       0.94      0.95      0.94      7266
weighted avg       0.95      0.95      0.95      7266

0.9454995871180842
